<a href="https://colab.research.google.com/github/hvarS/CS60075-Team28-Task-1/blob/main/experiment_sanket.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
import gensim.downloader
from gensim.models import KeyedVectors
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize 
import nltk
import pandas as pd 
import numpy as np
import os 

In [3]:
word_vector = gensim.downloader.load('word2vec-google-news-300')

[=================================================-] 99.3% 1651.2/1662.8MB downloaded


In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
%cd /content/drive/MyDrive/nlp_project

/content/drive/MyDrive/nlp_project


In [11]:
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [17]:
 !pip install syllables

In [41]:
def add_single_features(name,isTrain) :
  df = pd.read_csv("data/preprocessed/" +name )

  frequency_of_word = {}
  for i in range(len(df)) :
    word = df.at[i,"token"]
    if word in frequency_of_word :
      frequency_of_word[word]+=1
    else :
      frequency_of_word[word] = 1



  def vowel_count(str): 
    count = 0
    vowel = set("aeiouAEIOU") 
    for alphabet in str: 
      if alphabet in vowel: 
          count = count + 1
    return count 


  def contain_numeral(str) :
    num = "0123456789"
    for alphabet in str: 
      if alphabet in num: 
          return True
    return False


  token_length=[]
  token_syllable= []
  token_vowels = []
  token_frequency = []
  token_contain_numeral = []
  nearest_class = []
  # print(df.head())
  for i in range(len(df)) :
    
    word = df.at[i,"token"]
    if type(word) is not str :
      token_length.append(0)
      token_syllable.append(0)
      token_frequency.append(0)
      token_vowels.append(0)
      token_contain_numeral.append(False)
      if isTrain :
        nearest_class.append(round(4*df.at[i,"complexity"])+1)   
      
      continue
    token_length.append(len(word))
    import syllables # pip install syllables
    token_syllable.append(syllables.estimate(word))
    token_vowels.append(vowel_count(word))
    token_frequency.append(frequency_of_word[word])
    token_contain_numeral.append(contain_numeral(word))

    # print(type(df.at[i,"complexity"]))
    if isTrain :
      nearest_class.append(round(4*df.at[i,"complexity"])+1)

  df['token_length'] = token_length
  df['token_syllable'] = token_syllable
  df['token_vowels'] = token_vowels
  df['token_frequency'] = token_frequency
  df['token_contain_numeral'] = token_contain_numeral
  if isTrain :
    df['nearest_class'] = nearest_class
  df.to_csv("data/added_features/"+name,index=False)


In [42]:
add_single_features("lcp_single_test_preprocessed.csv",False)
add_single_features("lcp_single_train_preprocessed.csv",True)
add_single_features("lcp_multi_test_preprocessed.csv",False)
add_single_features("lcp_multi_train_preprocessed.csv",True)

models on single

In [50]:
data_folder = "data/added_features"
df_Train = pd.read_csv(os.path.join(data_folder,"lcp_single_train_preprocessed.csv"),index_col=0)
df_Test = pd.read_csv(os.path.join(data_folder,"lcp_single_test_preprocessed.csv"),index_col=0)
df_Test

,corpus,sentence,token,token_length,token_syllable,token_vowels,token_frequency,token_contain_numeral
id,,,,,,,,
39TX062QX1OHFOH8FUL76K5L7D3X3S,bible,speak much prince world come nothing,prince,6,2,2,5,False
3CIS7GGG65JS8I3AZ9RG54AE4MUUEA,bible,house shall turned others field wife together ...,inhabitant,10,4,4,3,False
379OL9DBSSESUVWY1Z8JGBFG9BTY92,bible,stranger terrible nation cut left mountain val...,bough,5,1,2,2,False
3DFYDSXB2W00JYP2DA272KN69UQUJV,bible,sharpen tongue like sword aim arrow deadly word,arrow,5,2,2,3,False
31YWE12TE0CZG7IVH6OXJ1H1CFPX7X,bible,obey leader submit watch behalf soul give acco...,account,7,2,3,4,False
...,...,...,...,...,...,...,...,...
3B0MCRZMBRUJD4K4R18XPAMJAE2PPW,europarl,accession european community convention concer...,convention,10,3,4,4,False
3IKMEYR0LWVMA6MICBSDXGSR5U2K21,europarl,question oral answer commission stress test eu...,liberal,7,3,3,6,False
37M4O367VJI9ZR58F67RA0N7E885MA,europarl,received legal opinion parliament legal servic...,comparison,10,4,4,1,False


In [69]:
stat_feature = np.array(df_Train[['token_length','token_syllable','token_vowels','token_frequency','token_contain_numeral']])

embed_word = np.array(list(df_Train['token'].apply(lambda x:word_vector[x] if x in word_vector else word_vector['unk'])))
embed_sentence = np.array(list(df_Train['sentence'].apply
    (
    lambda x:
    sum([word_vector[w] if w in word_vector else word_vector['unk'] for w in x.split()])/len(x.split())
    )
                              )
                         )
Train_Vector = np.hstack((stat_feature,.5*embed_word+.5*embed_sentence))
Train_Vector.shape

(7662, 305)

In [68]:
stat_feature = np.array(df_Test[['token_length','token_syllable','token_vowels','token_frequency','token_contain_numeral']])

embed_word = np.array(list(df_Test['token'].apply(lambda x:word_vector[x] if x in word_vector else word_vector['unk'])))
embed_sentence = np.array(list(df_Test['sentence'].apply
    (
    lambda x:
    sum([word_vector[w] if w in word_vector else word_vector['unk'] for w in x.split()])/len(x.split())
    )
                              )
                         )
Test_Vector = np.hstack((stat_feature,.5*embed_word+.5*embed_sentence))
Test_Vector.shape

(917, 305)

In [70]:
submission_folder = "predictions/single_sanket"


In [72]:
import os
import pandas as pd
import numpy as np
import gensim.downloader
from gensim.models import KeyedVectors
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize 

from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor


In [75]:
# Linear Regression
reg = LinearRegression().fit(Train_Vector, np.array(df_Train['complexity']))
y_pred = reg.predict(Test_Vector)

pred = pd.DataFrame({"ID":df_Test.index, "complexity":y_pred})
pred.to_csv(submission_folder+"/linear_regression_baseline.csv", index=False, header=False)


In [77]:

# SVM regressor
reg = SVR().fit(Train_Vector, np.array(df_Train['complexity']))
y_pred = reg.predict(Test_Vector)

pred = pd.DataFrame({"ID":df_Test.index, "complexity":y_pred})
pred.to_csv(submission_folder+"/SVM_baseline.csv", index=False, header=False)

In [78]:

# Gradient Boosting
reg = GradientBoostingRegressor().fit(Train_Vector, np.array(df_Train['complexity']))
y_pred = reg.predict(Test_Vector)

pred = pd.DataFrame({"ID":df_Test.index, "complexity":y_pred})
pred.to_csv(submission_folder+"/gradient_boosting_baseline.csv", index=False, header=False)

In [79]:

# MLP Regressor
regr = MLPRegressor(hidden_layer_sizes=(150)).fit(Train_Vector, np.array(df_Train['complexity']))
y_pred = reg.predict(Test_Vector)

pred = pd.DataFrame({"ID":df_Test.index, "complexity":y_pred})
pred.to_csv(submission_folder+"/MLP_baseline.csv", index=False, header=False)